# NBA DX project

## Scraper

In [1]:
from bs4 import BeautifulSoup, Comment
import pathlib
import json
import time
from scraper.scraper import scrap_functions
from scraper.scraper import get_player_seasons
from scraper.url import get_seasons_url, get_players_url, get_player_soup
from tqdm import tqdm
import pandas as pd

from const import *
from utils import *

In [2]:
# Download HTML to scrape
seasons = get_seasons_url(START_YEAR, END_YEAR)

In [3]:
urls = []
# loop through the seasons
for season in seasons:
    player_urls = get_players_url(season)

    # add new player_urls to urls
    urls.extend(player_urls)

urls = list(set(urls))

In [4]:
players_data = {}
for year in range(START_YEAR, END_YEAR + 1):
    players_data[str(year)] = {}

# loop through first 10 urls
for url in urls[:10]:
    # get the player's HTML page
    soup_file = get_player_soup(url)

    seasons = get_player_seasons(soup_file)
    print(seasons)

    for season_str in seasons:
        # Scrape HTML
        player = {}
        player[URL] = url
        
        year = int(season_str.split('-')[0]) + 1
        if year >= START_YEAR and year <= END_YEAR:
            player[SEASON] = season_str
            
            for key, function in scrap_functions.items():
                player[key] = function(soup_file, season=season_str)

            players_data[str(year)][url] = player


# save each season to a different JSON file
for year in range(START_YEAR, END_YEAR + 1):
    destination = OUTPUT_DIR / f'{year-1}-{str(year)[2:]}.json'
    with open(str(destination), 'w') as f:
        print(f'Saving data to {destination}')
        json.dump(players_data[str(year)], f, indent=4, default=str)

['2017-18', '2016-17', '2015-16', '2019-20', '2018-19', '2020-21']
2017-18
2016-17
2015-16
2019-20
2018-19
['2007-08', '2016-17', '2017-18', '2004-05', '2015-16', '2008-09', '2014-15', '2010-11', '2012-13', '2011-12', '2013-14', '2002-03', '2003-04', '2006-07', '2009-10', '2005-06']
2016-17
2017-18
2015-16
2014-15
2010-11
2012-13
2011-12
2013-14
2009-10
['2007-08', '2008-09', '2010-11', '2012-13', '2011-12', '2006-07', '2009-10', '2005-06']
2010-11
2012-13
2011-12
2009-10
['2007-08', '2016-17', '2004-05', '2015-16', '2008-09', '2014-15', '2010-11', '2012-13', '2011-12', '2013-14', '2003-04', '2006-07', '2009-10', '2005-06']
2016-17
2015-16
2014-15
2010-11
2012-13
2011-12
2013-14
2009-10
['2008-09', '2010-11', '2013-14', '2012-13', '2011-12', '2009-10']
2010-11
2013-14
2012-13
2011-12
2009-10
['2019-20', '2022-23', '2018-19', '2020-21', '2021-22']
2019-20
2018-19
['2017-18', '2016-17', '2007-08', '2015-16', '2008-09', '2014-15', '2019-20', '2013-14', '2012-13', '2011-12', '2018-19', '20